In [1]:
import sys
sys.path.append( '../' )

In [2]:
import numpy as np
import cs_vqe as c
import ast
import os
import scipy as sp
from copy import deepcopy

In [3]:
from openfermion import qubit_operator_sparse
import quchem.Misc_functions.conversion_scripts as conv_scr
from openfermion.ops import QubitOperator

In [4]:
# with open("hamiltonians.txt", 'r') as input_file:
#     hamiltonians = ast.literal_eval(input_file.read())
    
cwd = os.getcwd()
working_dir = os.path.dirname(cwd)
data_dir = os.path.join(working_dir, 'data')
data_hamiltonians_file = os.path.join(data_dir, 'hamiltonians.txt')


with open(data_hamiltonians_file, 'r') as input_file:
    hamiltonians = ast.literal_eval(input_file.read())

In [5]:
for key in hamiltonians.keys():
    print(f"{key: <25}     n_qubits:  {hamiltonians[key][1]:<5.0f}")

H2-S1_STO-3G_singlet          n_qubits:  18   
C1-O1_STO-3G_singlet          n_qubits:  16   
H1-Cl1_STO-3G_singlet         n_qubits:  16   
H1-Na1_STO-3G_singlet         n_qubits:  16   
H2-Mg1_STO-3G_singlet         n_qubits:  17   
H1-F1_3-21G_singlet           n_qubits:  18   
H1-Li1_3-21G_singlet          n_qubits:  18   
Be1_STO-3G_singlet            n_qubits:  5    
H1-F1_STO-3G_singlet          n_qubits:  8    
H1-Li1_STO-3G_singlet         n_qubits:  8    
Ar1_STO-3G_singlet            n_qubits:  13   
F2_STO-3G_singlet             n_qubits:  15   
H1-O1_STO-3G_singlet          n_qubits:  8    
H2-Be1_STO-3G_singlet         n_qubits:  9    
H2-O1_STO-3G_singlet          n_qubits:  10   
H2_3-21G_singlet              n_qubits:  5    
H2_6-31G_singlet              n_qubits:  5    
H3-N1_STO-3G_singlet          n_qubits:  13   
H4-C1_STO-3G_singlet          n_qubits:  14   
Mg1_STO-3G_singlet            n_qubits:  13   
N2_STO-3G_singlet             n_qubits:  15   
Ne1_STO-3G_si

In [6]:
data_csvqe_results_file = os.path.join(data_dir, 'csvqe_results.txt')
with open(data_csvqe_results_file, 'r') as input_file:
    csvqe_results = ast.literal_eval(input_file.read())

for key in csvqe_results.keys():
    print(f"{key: <25}     n_qubits:  {hamiltonians[key][1]:<5.0f}")

H3_STO-3G_singlet_1+          n_qubits:  3    
Be1_STO-3G_singlet            n_qubits:  5    
H2_3-21G_singlet              n_qubits:  5    
H2_6-31G_singlet              n_qubits:  5    
H1-He1_3-21G_singlet_1+       n_qubits:  6    
H1-F1_STO-3G_singlet          n_qubits:  8    
H1-Li1_STO-3G_singlet         n_qubits:  8    
H1-O1_STO-3G_singlet          n_qubits:  8    
H2-Be1_STO-3G_singlet         n_qubits:  9    
H3_3-21G_singlet_1+           n_qubits:  9    
H2-O1_STO-3G_singlet          n_qubits:  10   
H3-N1_STO-3G_singlet          n_qubits:  13   
Mg1_STO-3G_singlet            n_qubits:  13   
H4-C1_STO-3G_singlet          n_qubits:  14   
H4-N1_STO-3G_singlet_1+       n_qubits:  14   
F2_STO-3G_singlet             n_qubits:  15   
N2_STO-3G_singlet             n_qubits:  15   
O2_STO-3G_singlet             n_qubits:  15   
C1-O1_STO-3G_singlet          n_qubits:  16   
H1-Cl1_STO-3G_singlet         n_qubits:  16   
H1-Na1_STO-3G_singlet         n_qubits:  16   
H2-Mg1_STO-3G

In [7]:
# mol_k
mol_key ='H3_STO-3G_singlet_1+'
# mol_key ='H2-Be1_STO-3G_singlet'
# mol_key = 'H1-Li1_STO-3G_singlet'


In [8]:
print(f"n_qubits:  {hamiltonians[mol_key][1]}")

n_qubits:  3


# Get non-contextual H

In [9]:
ham = hamiltonians[mol_key][2]
ham

{'III': -1.7512307459285525,
 'IIX': -0.023568152336180023,
 'IIZ': -0.017109477140260287,
 'IXI': 0.02356815233618002,
 'IXZ': 0.02356815233617983,
 'IYY': -0.07195737217001562,
 'IZI': -0.017109477140260287,
 'IZX': -0.023568152336179825,
 'IZZ': 0.31270210682950855,
 'XII': 0.01872992170537467,
 'XIX': 0.023568139980123585,
 'XIZ': 0.01872992170537467,
 'XXI': -0.023568139980123585,
 'XXX': 0.03597868636603963,
 'XXZ': -0.023568139980123585,
 'XYY': -0.03597868636603963,
 'XZI': 0.01872992170537467,
 'XZX': 0.023568139980123585,
 'XZZ': 0.01872992170537467,
 'YIY': 0.023568139980123585,
 'YXY': 0.03597868636603963,
 'YYI': -0.023568139980123585,
 'YYX': 0.03597868636603963,
 'YYZ': -0.023568139980123585,
 'YZY': 0.023568139980123585,
 'ZII': -0.45436486525596403,
 'ZIX': -0.023568152336180023,
 'ZIZ': 0.37110605476609787,
 'ZXI': 0.02356815233618002,
 'ZXZ': 0.02356815233617983,
 'ZYY': -0.07195737217001562,
 'ZZI': 0.37110605476609804,
 'ZZX': -0.023568152336179825,
 'ZZZ': -0.2878

In [10]:
n_qubits = hamiltonians[mol_key][1]

ham_noncon = hamiltonians[mol_key][3]
Con_H = {}

for P in ham:
    if P in ham_noncon:
        continue
    else:
        Con_H[P]=ham[P]
        

## Testing contextuality

In [11]:
print('Is NONcontextual correct:', not c.contextualQ_ham(ham_noncon))
print('Is contextual correct:',c.contextualQ_ham(Con_H))

Is NONcontextual correct: True
Is contextual correct: True


# Classical part of problem!

Take $H_{nc}$ and split into:
- $Z$ = operators that completely comute with all operators in $S$
- $T$ = remaining operators in $S$
    - where $S = Z \cup T$  and $S$ is set of Pauli operators in $H_{nc}$
    
    
- We then split the set $T$ into cliques $C_{1}, C_{2}, ... , C_{|T|}$
    - all ops in a clique commute
    - ops between cliques anti-commute!

In [12]:
model = c.quasi_model(ham_noncon)
fn_form = c.energy_function_form(ham_noncon, model)

# Function form

$$R = G \cup \{ C_{i1} \;| \; i=1,2,...,N \}$$

- note q to do with $G$
- note r to do with $C_{i1}$

In [13]:
Energy_function = c.energy_function(fn_form)

In [14]:
ground_state_params = [list(hamiltonians[mol_key][5][1]), list(hamiltonians[mol_key][5][2])]
ground_state_params

[[-1, 1], [0.9799593338761482, -0.19919765046058108]]

In [15]:
## check

H_symbolic = conv_scr.Get_Openfermion_Hamiltonian(ham_noncon)
sparseH = qubit_operator_sparse(H_symbolic, n_qubits=n_qubits)

if sparseH.shape[0]<128:
    eig_vals, eig_vecs = sp.linalg.eigh(sparseH.todense())
    lowest_eigenvalue = min(eig_vals)
else:
    lowest_eigenvalue = sp.sparse.linalg.eigsh(sparseH, which='SA', k=1)[0][0]
    
np.isclose(Energy_function(*ground_state_params[0],*ground_state_params[1]),
           lowest_eigenvalue)

True

# Now need to rotate Hamiltonian!

We now have non contextual ground state: $(\vec{q}, \vec{r})$

In [16]:
ground_state_params

[[-1, 1], [0.9799593338761482, -0.19919765046058108]]

We can use this result - ground state of $H_{nc}$ -  as a classical estiamte of our ground state of the full Hamiltonian ($H = H_{c} + H_{nc}$)

However we can also obtain a quantum correction using $H_{c}$

By minimizing theenergy of the remaining terms in the Hamiltonian over the quantum states that are **consistent with the noncon-textual ground state**.

To do this we first rotate each $G_{j}$ and $\mathcal{A} = \sum_{i=1}^{N} r_{i}A_{i}$:

In [17]:
true_gs = hamiltonians[mol_key][4]
true_gs

-2.916018490268454

In [18]:
order = csvqe_results[mol_key][-1] # <-- order specified from greedy run by Will
qubit_removal_order=deepcopy(order)
qubit_removal_order

[0, 2, 1]

# Rotate full Hamiltonian to basis with diagonal noncontextual generators!

In [19]:
from cs_vqe_with_SeqRot import diagonalize_epistemic_dictionary_generator

In [20]:
(mapping_GuA_to_singleZ_with_ep_exp_vals, 
 R_sk_OP_list,
GuA_full_rotated) = diagonalize_epistemic_dictionary_generator(model, 
                                                            fn_form, 
                                                            ground_state_params, 
                                                            check_reduction=False)

In [21]:
from cs_vqe_with_SeqRot import get_wills_order

qubit_removal_order=deepcopy(order)

actual_qubit_removal_order = get_wills_order(GuA_full_rotated, qubit_removal_order)
actual_qubit_removal_order

[0, 2, 1]

In [22]:
mapping_GuA_to_singleZ_with_ep_exp_vals

{'ZZI': {'single_Z': 'ZII', 'noncon_gs_exp_val': -1, 'do_unitary_part': False},
 'IZZ': {'single_Z': 'IZI', 'noncon_gs_exp_val': 1, 'do_unitary_part': False},
 'ZII': {'single_Z': 'IIZ', 'noncon_gs_exp_val': 1, 'do_unitary_part': True}}

In [23]:
from cs_vqe_with_SeqRot import diagonalize_epistemic_SeqRot

In [24]:
(rotations, 
 do_unitary_part, 
 ep_state_trans,
 GuA) = diagonalize_epistemic_SeqRot(model, mapping_GuA_to_singleZ_with_ep_exp_vals, [2])
GuA

['ZII']

In [25]:
from cs_vqe_with_SeqRot import get_reduced_hamiltonian_by_qubits_fixed

In [26]:
H_reduced = get_reduced_hamiltonian_by_qubits_fixed(ham,
                                             model, 
                                             mapping_GuA_to_singleZ_with_ep_exp_vals,
                                             [0],
                                             R_sk_OP_list)
# H_reduced

In [27]:
from cs_vqe_with_SeqRot import get_reduced_hamiltonians_by_order 


H_reduced_list = get_reduced_hamiltonians_by_order(ham,
                                             model, 
                                             mapping_GuA_to_singleZ_with_ep_exp_vals,
                                             actual_qubit_removal_order, # <-- actual GOOD order!
                                             R_sk_OP_list)
H_reduced_list

[{'': (-2.9032129187167244+0j)},
 {'I': (-2.1802002532456592+0j),
  'Z': (0.7230126654710651+0j),
  'X': (-0.1350107578751797+0j)},
 {'II': (-1.438528639099044+0j),
  'IZ': (-0.034218954280520575+0j),
  'IX': -0.07195737217001562,
  'XI': (0.03745984341074934+0j),
  'XZ': (0.03745984341074934+0j),
  'XX': (-0.07195737273207926+0j),
  'YY': (-0.07195737273207926+0j),
  'ZI': (-0.7422123035331922+0j),
  'ZZ': (0.7422121095321959+0j),
  'ZX': -0.07195737217001562},
 {'III': -1.7512307459285525,
  'IIX': -0.023568152336180023,
  'IIZ': -0.017109477140260287,
  'IXI': 0.02356815233618002,
  'IXZ': 0.02356815233617983,
  'IYY': -0.07195737217001562,
  'IZI': -0.017109477140260287,
  'IZX': -0.023568152336179825,
  'IZZ': 0.31270210682950855,
  'XII': 0.01872992170537467,
  'XIX': 0.023568139980123585,
  'XIZ': 0.01872992170537467,
  'XXI': -0.023568139980123585,
  'XXX': 0.03597868636603963,
  'XXZ': -0.023568139980123585,
  'XYY': -0.03597868636603963,
  'XZI': 0.01872992170537467,
  'XZX':

In [28]:
qubit_removal_order=deepcopy(order)
qubit_removal_order
print(qubit_removal_order)
print()

red_Hs = c.get_reduced_hamiltonians(ham,
                           model,
                           fn_form,
                           ground_state_params,
                           qubit_removal_order)
red_Hs

[0, 2, 1]



[{'': -2.903212918716724},
 {'I': -2.180200253245659,
  'Z': -0.7230126654710651,
  'X': -0.13501075787517966},
 {'II': -1.438528639099044,
  'ZZ': -0.01919944406113084,
  'ZX': -0.07733163434390385,
  'XI': -0.07195737217001562,
  'XX': 0.051042862665487465,
  'XZ': 0.06305338570516404,
  'YY': -0.03745984341074934,
  'IX': 0.07733164795674143,
  'IZ': 0.7416716141466151,
  'ZI': -0.7422121095321959},
 {'III': -1.7512307459285525,
  'ZXI': 0.023568152336180023,
  'ZZZ': 0.00959972203056542,
  'ZZX': 0.03866581717195192,
  'YYI': -0.02356815233618002,
  'XXZ': -0.027790548973821074,
  'XXX': 0.018401098184557648,
  'XII': -0.07195737217001562,
  'ZIZ': -0.00959972203056542,
  'ZIX': -0.03866581717195192,
  'IXZ': 0.02779054897382107,
  'IXX': -0.018401098184557648,
  'IZI': -0.31270210682950855,
  'XZX': -0.01835456159795137,
  'XZZ': 0.003730956397021274,
  'YYX': 0.02779053932668584,
  'YYZ': 0.018401112754287855,
  'YIY': -0.01872992170537467,
  'ZXX': -0.02779053932668584,
  'ZXZ':

In [29]:
len(H_reduced_list) == len(red_Hs)

True

In [30]:
E_list_one =[]
for H in red_Hs:
    H = {P_key: coeff.real for P_key, coeff in H.items() if not np.isclose(coeff.real,0)}
    
    H_mat = conv_scr.Get_Openfermion_Hamiltonian(H)
    sparseH = qubit_operator_sparse(H_mat, n_qubits=hamiltonians[mol_key][1])
    E = sp.sparse.linalg.eigsh(sparseH, which='SA', k=1)[0][0]
    print(E)
    print(len(H))
    print()
    E_list_one.append(E)

-2.9032129187167244
1

-2.9157104293639815
3

-2.9160184902684563
10

-2.916018490268449
35



In [31]:
E_list_two =[]
for H in H_reduced_list:
    H = {P_key: coeff.real for P_key, coeff in H.items() if not np.isclose(coeff.real,0)}
    H_mat = conv_scr.Get_Openfermion_Hamiltonian(H)
    sparseH = qubit_operator_sparse(H_mat, n_qubits=hamiltonians[mol_key][1])
    E = sp.sparse.linalg.eigsh(sparseH, which='SA', k=1)[0][0]
    print(E)
    print(len(H))
    print()
    E_list_two.append(E)

-2.9032129187167244
1

-2.915710429363982
3

-2.9160184902684527
10

-2.9160184902684523
34



In [32]:
np.array(E_list_two) - np.array(E_list_one)

array([ 0.00000000e+00, -4.44089210e-16,  3.55271368e-15, -3.10862447e-15])

In [33]:
# cs_VQE new
np.array(E_list_two) - true_gs

array([1.28055716e-02, 3.08060904e-04, 1.33226763e-15, 1.77635684e-15])

In [34]:
# cs_VQE will
np.array(E_list_one) - true_gs

array([ 1.28055716e-02,  3.08060904e-04, -2.22044605e-15,  4.88498131e-15])

In [35]:
# look for: do_unitary_part=TRUE
mapping_GuA_to_singleZ_with_ep_exp_vals


{'ZZI': {'single_Z': 'ZII', 'noncon_gs_exp_val': -1, 'do_unitary_part': False},
 'IZZ': {'single_Z': 'IZI', 'noncon_gs_exp_val': 1, 'do_unitary_part': False},
 'ZII': {'single_Z': 'IIZ', 'noncon_gs_exp_val': 1, 'do_unitary_part': True}}

In [36]:
'IIIIIIIIZ'.index('Z')

8

In [37]:
actual_qubit_removal_order

[0, 2, 1]